In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import jieba
import numpy as np
train_first = pd.read_csv('E:\\data mining\\dataset\\Reputation-evaluation-of-scenic-spots\\chusai\\train_first.csv')
train_second = pd.read_csv('E:\\data mining\\dataset\\Reputation-evaluation-of-scenic-spots\\fusai\\train_second.csv')
train = pd.concat([train_first,train_second])
test = pd.read_csv('E:\\data mining\\dataset\\Reputation-evaluation-of-scenic-spots\\fusai\\predict_second.csv')
train.columns = ['id', 'discuss', 'score']
test.columns = ['id', 'discuss']

In [2]:
stopword = ['【','】','"','\'','<','>','的','了','“','”','‘','’',' ']

In [3]:
import re
patt = re.compile('[\u4e00-\u9fa5|,|.|?|!|‘|’|“|”|【|】]')

In [4]:
train['discuss'] = train['discuss'].map(lambda x : ''.join(patt.findall(x)))
test['discuss'] = test['discuss'].map(lambda x : ''.join(patt.findall(x)))

In [5]:
train['discuss'] = train['discuss'].map(lambda x : [k for k in jieba.lcut(x) if k not in stopword])
test['discuss'] = test['discuss'].map(lambda x :[k for k in jieba.lcut(x) if k not in stopword])
all_data = pd.concat([train,test])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1.WIN\AppData\Local\Temp\jieba.cache
Loading model cost 1.845 seconds.
Prefix dict has been built succesfully.


In [6]:
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
dic = Dictionary(all_data['discuss'].values)
corpus = [dic.doc2bow(x) for x in all_data['discuss'].values]
model = Word2Vec(sg=1,size=64,window=4,negative=4)
model.build_vocab(all_data['discuss'].values)
model.train(all_data['discuss'].values,total_examples=model.corpus_count,epochs=model.iter)

E:\anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


(25569103, 29022805)

In [7]:
w2v = []
for line in all_data['discuss'].values:
    temp = np.zeros((1,64))
    i=0
    for w in line:
        try:
            i+=1
            temp+=np.array(model[w])
        except Exception:
            continue
    w2v.append(temp[0]/i)
feature=w2v

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [8]:
score = train.score.values
from sklearn.model_selection import train_test_split
import lightgbm as lgb
test_real_list = []
seeds = [2016,2017,2018,2019,2020,2021,20222,2023,2024,2025]
test_csr = feature[len(train):]
for seed in seeds:
    X_train, X_test, y_train, y_test = train_test_split(feature[:len(train)], score, test_size=0.2,random_state=seed)
    d_train = lgb.Dataset(X_train, label=y_train,silent=False)
    d_test = lgb.Dataset(X_test,label=y_test,silent=False)
    params = {
        'application': 'regression',
        'num_leaves': 64,
        'metric': 'RMSE',
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'reg_alpha':5,
        'reg_lambda':5,
        'learning_rate':0.08
    }
    model = lgb.train(params, train_set=d_train, num_boost_round=15000,verbose_eval=5000,valid_sets=[d_train,d_test])
    re_real = model.predict(test_csr)
    test_real_list.append(re_real)
    print('finish one')

[5000]	training's rmse: 0.285992	valid_1's rmse: 0.623875
[10000]	training's rmse: 0.222126	valid_1's rmse: 0.621037
[15000]	training's rmse: 0.209477	valid_1's rmse: 0.621054
finish one
[5000]	training's rmse: 0.285756	valid_1's rmse: 0.622896
[10000]	training's rmse: 0.22318	valid_1's rmse: 0.620054
[15000]	training's rmse: 0.2103	valid_1's rmse: 0.619797
finish one
[5000]	training's rmse: 0.285681	valid_1's rmse: 0.618507
[10000]	training's rmse: 0.222982	valid_1's rmse: 0.616073
[15000]	training's rmse: 0.210346	valid_1's rmse: 0.616019
finish one
[5000]	training's rmse: 0.286299	valid_1's rmse: 0.617045
[10000]	training's rmse: 0.223582	valid_1's rmse: 0.614106
[15000]	training's rmse: 0.211294	valid_1's rmse: 0.613974
finish one
[5000]	training's rmse: 0.28542	valid_1's rmse: 0.620709
[10000]	training's rmse: 0.221378	valid_1's rmse: 0.617867
[15000]	training's rmse: 0.208682	valid_1's rmse: 0.61778
finish one
[5000]	training's rmse: 0.286695	valid_1's rmse: 0.618406
[10000]	trai

In [9]:
import numpy as np
test_real_avg = np.sum(test_real_list,axis=0)/10
test['score'] = test_real_avg
test['score'] = test['score'].map(lambda x: 5 if x > 4.7 else x)
test[['id','score']].to_csv('new_try.csv',index=False)

In [ ]:
len(test_real_list)